In [19]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
from datetime import date
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, GeoData, LayersControl

import ipywidgets as widgets 
from ipywidgets import AppLayout, Button, Layout, Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, DatePicker, Output, VBox, HBox, Text, SelectMultiple, Combobox, Accordion
from ipywidgets.embed import embed_data
from IPython.display import clear_output, HTML, display, Image

import threading
import time
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
import datetime
import itertools

from app import *

In [20]:
# HEADER
header = widgets.HTML("<h1>OSM Data Change Tracker</h1>", layout=Layout(height='auto'))
header.style.text_align='center'

footer = widgets.HTML('<h5 style="color:DodgerBlue;"> Source: OpenStreet Map via Overpass API </h5>', layout=Layout(height='auto'))

# DATE PICKER
start_dt = DatePicker(layout=Layout(flex='1 1 0%', width='auto'), disabled=False)
end_dt = DatePicker(layout=Layout(flex='1 1 0%', width='auto'), disabled=False)

# Map coord 
map_coord = widgets.Text(
    value= '(-51.71312453127716, -59.65015951499032)',
    description='Map Center:',
    disabled=False, style=dict(description_width='initial'),
    continuous_update=False
    )

# Map Zoom
map_zoom = widgets.IntText(
    value=15,
    description='Zoom:',
    disabled=False,
    continuous_update=False
)

# BBOX 
bbox_coord = widgets.Text(
    value= '(37.22828,131.85043,37.25282, 131.88316)',
    description='Bbox Coord:',
    disabled=False, style=dict(description_width='initial')
    )

# Filenam
input_filename = widgets.Text(
    value='sample.csv',
    placeholder='filename',
    description='Save File as:',
    disabled=False, style=dict(description_width='initial')
    )

# SPECIFY INTERVAL
hourly_slider = widgets.IntSlider(
    value=24,
    min=1,
    max=200,
    description='Hourly Interval:'
)

# REQUEST BUTTON 
Request_button = widgets.Button(
    description='Download Data',
    disabled=False,
    button_style='success',
    icon='table', layout = Layout(width='200px', height='auto'),
    style=dict(description_width='initial'))


# BBOX Refresh button
bbox_button = widgets.Button(
    description='Update',
    disabled=False,
    button_style='warning',
    icon='arrow-rotate-right', layout = Layout(width='80px', height='auto'),
    style=dict(description_width='initial'))

progress_search = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, 
    description='Progress',
    bar_style='info',
    style={'bar_color': '#94b79f'})
Complete_msg = widgets.HTML()


In [21]:
import plotly.graph_objects as go
import dtale

In [22]:
def checkPivot_ChangeType(dff):
    if 'create' not in dff:
        dff['create'] = 0
    elif 'delete' not in dff:
        dff['delete'] = 0 
    elif 'modify' not in dff: 
        dff['modify'] = 0 
    return dff 

In [23]:
def checkPivot_OsmType(dff):
    if 'node' not in dff:
        dff['node'] = 0
    elif 'relation' not in dff:
        dff['relation'] = 0 
    elif 'way' not in dff: 
        dff['way'] = 0 
    return dff 


In [24]:
def getGeomTrueDF(output_csv_name):
    geom_df = is_geometryEdit(output_csv_name)
    geomTrue = geom_df[geom_df['is_geometry_edit'] == True]
    return geomTrue

In [25]:
from ipyleaflet import Map, WKTLayer, MarkerCluster, LayerGroup


In [37]:
from ipyleaflet import Map, WKTLayer

def createGeomEditMap(map_center, map_zoom, output_csv_name):

    terrain_base = basemap_to_tiles(basemaps.Stamen.Toner)

    m = Map(layers=(terrain_base, ), center=map_center, zoom=8)
    marker_list = [] 
    geom_df = getGeomTrueDF(output_csv_name).reset_index(drop=True)
    for i, row in geom_df.iterrows():
        if 'POLYGON' in row['old_geom']:
            w_oldlayer = WKTLayer(
                wkt_string=row['old_geom'],
                style={"fillColor": "red", "color":"red"},
            )
            w_newlayer = WKTLayer(
                wkt_string=row['new_geom'],
                style={"fillColor": "green", "color":"green"},
            )
            m.add_layer(w_oldlayer)
            m.add_layer(w_newlayer)
        elif 'POINT' in row['old_geom']:
            w_oldlayer = WKTLayer(
                wkt_string=row['old_geom'],
                style={"fillColor": "red", "color":"red"},
            )
            w_newlayer = WKTLayer(
                wkt_string=row['new_geom'],
                style={"fillColor": "green", "color":"green"},
            )
            marker_list.append(w_newlayer)
            marker_list.append(w_oldlayer)

    # layer_group = LayerGroup(layers=(marker_list))
    marker_cluster = MarkerCluster(
        markers=(marker_list),
    )

    m.add_layer(marker_cluster)

    m.layout.width='auto'
    m.layout.height='600'
    return m 

In [38]:
def tab1_on_value_change(change):
    clear_output()
    feature_collection = getEmptyFeatureCollection()
    tab1 = HBox(children=[VBox(children=[tab1_header, HBox(children=[Label(value='Start Date', layout=dict(height='auto')), 
                                                                 start_dt, Label(value='End Date', layout=dict(height='auto')), end_dt]), 
                                     hourly_slider, 
                                     HBox(children=[bbox_coord, bbox_button]), 
                                     input_filename, 
                                     Request_button, 
                                     HBox(children=[progress_search, Complete_msg]), footer]), 
                                     VBox(children=[HBox(children=[map_coord, map_zoom]), createMap(eval(map_coord.value), map_zoom.value, draw_control)])])
    display(tab1) 


def tab2_on_value_change(change):
  df_complete = pd.read_csv(os.path.join('../data', selection_filename.value)).drop(columns = ['Unnamed: 0'])
  df_unique = df_complete.drop_duplicates(subset = ["change_type", "osm_id","osm_type", "end_ds"]).reset_index(drop=True).drop(columns = ['start_ds', 'tag_key', 'tag_value', 'history'])
  with plot_output: 
    clear_output()
    df_byDay = df_unique.groupby(["end_ds", "change_type"])['osm_id'].count().reset_index(name='count')
    df_byDay['end_ds'] = pd.to_datetime(df_byDay['end_ds']).dt.date
    df_byDay['end_ds'] = df_byDay['end_ds'].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    df1 = df_byDay.pivot(index='end_ds', columns='change_type', values='count').fillna(0)
    df1 = df1.reset_index()
    df1 = checkPivot_ChangeType(df1)
    df1 = checkPivot_ChangeType(df1)
    df1 = checkPivot_ChangeType(df1)

    df_byDay_osmType = df_unique.groupby(["end_ds", "osm_type"])['osm_id'].count().reset_index(name='count')
    df_byDay_osmType['end_ds'] = pd.to_datetime(df_byDay_osmType['end_ds']).dt.date
    df_byDay_osmType['end_ds'] = df_byDay_osmType['end_ds'].apply(lambda x: x.strftime('%Y-%m-%d'))
    df2 = df_byDay_osmType.pivot(index='end_ds', columns='osm_type', values='count').fillna(0)
    df2 = df2.reset_index()
    df2 = checkPivot_OsmType(df2)
    df2 = checkPivot_OsmType(df2)
    df2 = checkPivot_OsmType(df2)

    fig = go.Figure(go.Bar(x=df1['end_ds'], y=df1['create'], marker_color= 'green', opacity=0.65, name='create'))
    fig.add_trace(go.Bar(x=df1['end_ds'], y=df1['delete'], marker_color= 'firebrick', opacity=0.65, name='delete'))
    fig.add_trace(go.Bar(x=df1['end_ds'], y=df1['modify'], marker_color= 'purple', opacity=0.65, name='modify'))
    fig.add_trace(go.Scatter(x=df2['end_ds'], y=df2['node'], opacity=0.85, marker_color='yellow', name='node'))
    fig.add_trace(go.Scatter(x=df2['end_ds'], y=df2['relation'], opacity=0.85, marker_color= 'orange', name='relation'))
    fig.add_trace(go.Scatter(x=df2['end_ds'], y=df2['way'], opacity=0.85, marker_color='steelblue', name='way'))
    fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})

    g1 = go.FigureWidget(data=fig,
                    layout=go.Layout(
                        title=dict(
                            text='Change Type'
                        ),
                        barmode='stack'
                    ))
    display(g1)

  with geomTable_output: 
    clear_output()
    geom_df = getGeomTrueDF(selection_filename.value).reset_index(drop=True)
    if len(geom_df) != 0:
      display(geom_df)
    else: 
      display(widgets.HTML(value='<h2>No Geometry Edits!</h2>'))


  with table_output:
    clear_output() 
    counter_df = osmIDCounter(os.path.join('../data', selection_filename.value)).head(10).reset_index(drop = True)
    # widget = qgrid.show_grid(counter_df)
    # display(widget)
    # display(dtale.show(counter_df))
    display(HTML(counter_df.to_html(render_links=True, escape=False)))

  with completetable_output:
    clear_output()
    display(dtale.show(df_complete))

  with map_output:
    clear_output()
    display(createMapDiff(eval(map_coord.value), map_zoom.value, draw_control))

  with geom_map_output:
    clear_output()
    display(createGeomEditMap(eval(map_coord.value), map_zoom, selection_filename.value))

In [39]:
def work(progress_search):
    Complete_msg.value = ""
    total = 100
    for i in range(total):
        time.sleep(0.3)
        progress_search.value = float(i+1)/total
    Complete_msg.value = f"<h4 style='color:MediumSeaGreen;'> Complete! </h4>"

def callback(wdgt):
    thread = threading.Thread(target=work, args=(progress_search,))
    display(progress_search)
    thread.start()
    retrieveOSMDATA(hourly_slider.value, start_dt.value, end_dt.value, input_filename.value, bbox_coord.value)
    
def updateBBOX(wdgt):
  if len(feature_collection['features']) != 0: 
    coord_list = feature_collection['features'][0]['geometry']['coordinates'][0]
    minlon = coord_list[0][1]
    minlat = coord_list[0][0]
    maxlon = coord_list[1][1]
    maxlat = coord_list[2][0]
    bbox_tuple = (minlon, minlat, maxlon, maxlat)
    # bbox value 
    bbox_coord.value = str(bbox_tuple)
  else: 
    pass 

In [40]:
feature_collection = getEmptyFeatureCollection()
draw_control = getDrawControl()

Request_button.on_click(callback)
bbox_button.on_click(updateBBOX)
map_coord.observe(tab1_on_value_change, names='value')
map_zoom.observe(tab1_on_value_change, names='value')


In [45]:
plot_output = widgets.Output(layout = Layout(width='auto', height='100'))
table_output = widgets.Output(layout = Layout(width='400px', height='auto'))
geomTable_output = widgets.Output(layout = Layout(width='auto', height='100'))
map_output = widgets.Output(layout = Layout(width='400px', height='auto'))
geom_map_output = widgets.Output(layout = Layout(width='600px', height='auto'))
completetable_output = widgets.Output(layout = Layout(width='auto', height='auto'))

selection_filename = widgets.Dropdown(
    options=get_files(os.path.join('../data')),
    description='Select File:',
    style=dict(description_width='initial')
)

%matplotlib inline


selection_filename.observe(tab2_on_value_change, names='value')

In [46]:
tab1_header = widgets.HTML(value='<h2>OSM Change Downloader</h2>')
tab1 = HBox(children=[VBox(children=[tab1_header, HBox(children=[Label(value='Start Date', layout=dict(height='auto')), 
                                                                 start_dt, Label(value='End Date', layout=dict(height='auto')), end_dt]), 
                                     hourly_slider, 
                                     HBox(children=[bbox_coord, bbox_button]), 
                                     input_filename, 
                                     Request_button, 
          
                                     HBox(children=[progress_search, Complete_msg]), footer]), 
                                     VBox(children=[HBox(children=[map_coord, map_zoom]), createMap(eval(map_coord.value), map_zoom.value, draw_control)])])


In [47]:
tab2_header = widgets.HTML(value='<h2>Change Frequency</h2>')
table_header = widgets.HTML(value='<h2>Top 10 OSM IDs</h2>')

tab2 = HBox(children=[VBox(children=[tab2_header, selection_filename, HBox(children=[plot_output])])])
tab3 = VBox(children=[geom_map_output, geomTable_output])
tab4 = HBox(children=[table_output])
tab5 = completetable_output

In [48]:
tab_nest = widgets.Tab()
tab_nest.children = [tab1, tab2, tab3, tab4, tab5]
tab_nest.set_title(0, 'Downloader')
tab_nest.set_title(1, 'Change Count')
tab_nest.set_title(2, 'Geom Change')
tab_nest.set_title(3, 'Tag Change')
tab_nest.set_title(4, 'Complete Table')
tab_nest


In [34]:
df_complete = pd.read_csv(os.path.join('../data', selection_filename.value)).drop(columns = ['Unnamed: 0'])
df_unique = df_complete.drop_duplicates(subset = ["change_type", "osm_id","osm_type", "end_ds"]).reset_index(drop=True).drop(columns = ['start_ds', 'tag_key', 'tag_value', 'history'])
df_byDay = df_unique.groupby(["end_ds", "change_type"])['osm_id'].count().reset_index(name='count')
df_byDay_osmType = df_unique.groupby(["end_ds", "osm_type"])['osm_id'].count().reset_index(name='count')
df_byDay_osmType['end_ds'] = pd.to_datetime(df_byDay_osmType['end_ds']).dt.date
df_unique['end_ds'] = pd.to_datetime(df_unique['end_ds']).dt.date

In [35]:
import plotly.express as px


fig = px.bar(df_byDay, x = "end_ds", y = "count", color = 'change_type')
fig.show()

In [36]:
fig = px.line(df_byDay_osmType, x="end_ds", y="count",color="osm_type")
fig.show()